In [1]:
import os
import sys
import yaml
import json

with open("config.yaml", 'r') as yml_file:
    cfg = yaml.safe_load(yml_file)
    
#Get data path from config 
data_path = cfg['brian_laptop']['data_path']
mb_div = cfg['contants']['mb_div']

#Get dyype for meta file 
meta_data_file = os.path.join(data_path, cfg['data']['metadata'])
with open(meta_data_file) as f:
    meta_data = json.load(f)

In [2]:
import datatable as dt
print(dt.__version__)

0.9.0


## Bigger Data

55 million rows, 8 columns

In [3]:
%%time

nyc_taxi_fare_file = os.path.join(data_path, cfg['data']['kaggle_nyc'])
nyc_taxi_fare = dt.fread(nyc_taxi_fare_file)

CPU times: user 26.8 s, sys: 27.2 s, total: 54 s
Wall time: 20.8 s


In [5]:
%%time

print("data table memory(MB)", round(sys.getsizeof(nyc_taxi_fare)/mb_div,3))
print(nyc_taxi_fare.shape)
nyc_taxi_fare[:5,:]

data table memory(MB) 5690.856
(55423856, 8)
CPU times: user 398 µs, sys: 416 µs, total: 814 µs
Wall time: 1.59 ms


,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪
0,2009-06-15 17:26:21.0000001,4.5,2009-06-15 17:26:21 UTC,−73.8443,40.7213,−73.8416,40.7123,1
1,2010-01-05 16:52:16.0000002,16.9,2010-01-05 16:52:16 UTC,−74.016,40.7113,−73.9793,40.782,1
2,2011-08-18 00:35:00.00000049,5.7,2011-08-18 00:35:00 UTC,−73.9827,40.7613,−73.9912,40.7506,2
3,2012-04-21 04:30:42.0000001,7.7,2012-04-21 04:30:42 UTC,−73.9871,40.7331,−73.9916,40.7581,1
4,2010-03-09 07:51:00.000000135,5.3,2010-03-09 07:51:00 UTC,−73.9681,40.768,−73.9567,40.7838,1


In [6]:
%%time

#f is a frame proxy used to refer to the frame that is currently being operated on. 
nyc_taxi_fare[:, (dt.count(dt.f.fare_amount), 
                  dt.mean(dt.f.fare_amount),
                  dt.sd(dt.f.fare_amount),
                  dt.min(dt.f.fare_amount),
                  dt.median(dt.f.fare_amount),
                  dt.max(dt.f.fare_amount)),
              dt.by("passenger_count")]

CPU times: user 18.6 s, sys: 7.47 s, total: 26 s
Wall time: 4.19 s


,passenger_count,C0,C1,C2,C3,C4,C5
,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪
0,0,195416,9.01534,6.51254,−12.5,7.3,259
1,1,38337524,11.2164,18.5847,−300,8.5,75747
2,2,8175243,11.8384,34.4435,−176,8.5,93963.4
3,3,2432712,11.5406,9.81689,−300,8.5,500
4,4,1178852,11.7661,10.1969,−57.33,8.5,978
5,5,3929346,11.2084,9.34256,−67.5,8.5,1021.3
6,6,1174647,12.1262,10.1109,−60,9,477
7,7,15,31.7887,30.066,2.5,14.5,88
8,8,9,29.9811,34.0453,2.5,8,85


In [7]:
del(nyc_taxi_fare)

## PYDATATABLE INTERESTING BITS ##